In [1]:
#aysenur

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl

from torch.nn import functional as F
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger

import wandb

In [3]:
from DataPreparation import get_data

In [4]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 currency_list,
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage,
                 val_percentage,
                 test_percentage,
                 seq_len, 
                 ):
        self.currencies = currency_list
        self.n_currencies = len(self.currencies)
        self.x = torch.tensor(x[:self.n_currencies]).float()
        self.y = torch.tensor(y[:self.n_currencies]).long()
        self.seq_len = seq_len
        self.data_use_type = data_use_type
        
        
        #self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = int(len(self.x[0]) * test_percentage)
        self.train_size = len(self.x[0]) - self.val_size - self.test_size 
        
        self.train_mean = [self.x[i][:self.train_size].mean() for i in range(self.n_currencies)]
        self.train_std = [self.x[i][:self.train_size].std() for i in range(self.n_currencies)]
        
#         self.train_min = [self.x[i][:self.train_size].min() for i in range(n_currencies)]
#         self.train_max = [self.x[i][:self.train_size].max() for i in range(n_currencies)]
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size
  
        else:
            return self.test_size
        
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(self.n_currencies):
            window = self.x[i][index:index+self.seq_len]
            window = (window -self.train_mean[i]) / self.train_std[i]
            
            item[self.currencies[i] + "_window"] = window
            item[self.currencies[i] + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [5]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiheadAttention(nn.Module):
    
    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Stack all weight matrices 1...h together for efficiency
        # Note that in many implementations you see "bias=False" which is optional
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)
        
        self._reset_parameters()
        
        
    def _reset_parameters(self):
        # Original Transformer initialization, see PyTorch documentation
        nn.init.xavier_uniform_(self.qkv_proj.weight)
        self.qkv_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.o_proj.weight)
        self.o_proj.bias.data.fill_(0)
        
        
    def forward(self, x, mask=None, return_attention=False):
        batch_size, seq_length, embed_dim = x.size()
        qkv = self.qkv_proj(x)
        
        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)
        
        # Determine value outputs
        values, attention = scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, embed_dim)
        o = self.o_proj(values)
        
        if return_attention:
            return o, attention
        else:
            return o
        
class EncoderBlock(nn.Module):
    
    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()
        
        # Attention layer
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)
        
        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )
        
        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, x, mask=None):
        # Attention part
        attn_out = self.self_attn(x, mask=mask)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)
        
        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)
        
        return x
    
class TransformerEncoder(nn.Module):
    
    def __init__(self, num_layers, **block_args):
        super().__init__()
        self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])
        
    
    def forward(self, x, mask=None):
        for l in self.layers:
            x = l(x, mask=mask)
        return x
    
    
    def get_attention_maps(self, x, mask=None):
        attention_maps = []
        for l in self.layers:
            _, attn_map = l.self_attn(x, mask=mask, return_attention=True)
            attention_maps.append(attn_map)
            x = l(x)
        return attention_maps

In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model) 
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [7]:
class CosineWarmupScheduler(torch.optim.lr_scheduler._LRScheduler):
    
    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)
        
    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]
    
    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [8]:
class TradePredictor(pl.LightningModule): 
    def __init__(self, 
                 train_dataset,
                 val_dataset,
                 test_dataset,
                 calculate_loss_weights,
                 currencies,
                 window_size,
                 batch_size,
                 input_dim, model_dim, num_classes, num_heads, num_layers, 
                 lr,dropout=0.0, input_dropout=0.0,
                ):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()    
        self.num_tasks = len(currencies)

        self._create_model()
        
        self.f1_score = pl.metrics.F1(num_classes=self.hparams.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
        self.val_dl = DataLoader(val_dataset, batch_size=batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=batch_size)
        
        if self.hparams.calculate_loss_weights:
            loss_weights = []
            for i in range(self.num_tasks):
                train_labels = [int(train_dataset[n][self.hparams.currencies[i] +"_label"] )for n in range(train_dataset.__len__())]
                samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
                loss_weights.append((1 / samples_size) * sum(samples_size)/2)
            self.weights = loss_weights
        else:
            self.weights = None
            
        if self.weights != None:
            self.cross_entropy_loss = [nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) for weights in self.weights]
        else:
            self.cross_entropy_loss = [nn.CrossEntropyLoss() for _ in range(self.num_tasks)]
        
        self.cross_entropy_loss = torch.nn.ModuleList(self.cross_entropy_loss)
        
    def _create_model(self):
        # Input dim -> Model dim
        self.input_net = nn.Linear(self.hparams.input_dim, self.hparams.model_dim)
        
        # Positional encoding for sequences
        self.positional_encoding = PositionalEncoding(d_model=self.hparams.model_dim)
        # Transformer
        self.transformer = TransformerEncoder(num_layers=self.hparams.num_layers,
                                              input_dim=self.hparams.model_dim,
                                              dim_feedforward=2*self.hparams.model_dim,
                                              num_heads=self.hparams.num_heads,
                                              dropout=self.hparams.dropout)
        # Output classifier per sequence lement
        self.output_net = [nn.Sequential(
            nn.Linear(self.hparams.model_dim, self.hparams.model_dim),
            nn.LayerNorm(self.hparams.model_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(self.hparams.dropout),
            nn.Linear(self.hparams.model_dim, self.hparams.num_classes)
        ) ]* self.num_tasks
        
        self.output_net = torch.nn.ModuleList(self.output_net)
        
    
    def forward(self, x, i, mask=None, add_positional_encoding=True):
        """
        Inputs:
            x - Input features of shape [Batch, SeqLen, input_dim]
            mask - Mask to apply on the attention outputs (optional)
            add_positional_encoding - If True, we add the positional encoding to the input.
                                      Might not be desired for some tasks.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        x = self.transformer(x, mask=mask)
        x = x[:,-1,:]
        x = self.output_net[i](x)
        
        
        return x
    
    
    @torch.no_grad()
    def get_attention_maps(self, x, mask=None, add_positional_encoding=True):
        """
        Function for extracting the attention matrices of the whole Transformer for a single batch.
        Input arguments same as the forward pass.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        attention_maps = self.transformer.get_attention_maps(x, mask=mask)
        return attention_maps
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        
        # We don't return the lr scheduler because we need to apply it per iteration, not per epoch
        self.lr_scheduler = CosineWarmupScheduler(optimizer, 
                                                  warmup=50, 
                                                  max_iters = 80* self.train_dl.__len__())
        return optimizer
    
    
    def optimizer_step(self, *args, **kwargs):
        super().optimizer_step(*args, **kwargs)
        self.lr_scheduler.step() # Step per iteration

    def _calculate_loss(self, batch, mode="train"):
        
        loss = (torch.tensor(0.0, device="cuda:0", requires_grad=True) + \
                torch.tensor(0.0, device="cuda:0", requires_grad=True)) 
        for i in range(self.num_tasks):
            x, y = batch[self.hparams.currencies[i] + "_window"], batch[self.hparams.currencies[i] + "_label"]
  
            preds = self.forward(x, i, add_positional_encoding=True) # No positional encodings as it is a set, not a sequence!

            loss += self.cross_entropy_loss[i](preds, y) # Softmax/CE over set dimension
            
            acc = self.accuracy_score(torch.max(preds, dim=1)[1], y)
            self.log(self.hparams.currencies[i] + "_%s_acc" % mode, acc, on_step=False,  prog_bar=True, on_epoch=True)
            
            f1 = self.f1_score(torch.max(preds, dim=1)[1], y)
            self.log(self.hparams.currencies[i] + "_%s_f1" % mode, f1, on_step=False,  prog_bar=True, on_epoch=True)
        
        loss = loss / torch.tensor(self.num_tasks)
        self.log("%s_loss" % mode, loss, on_epoch=True, prog_bar=True)
        
        return loss
    
    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss
    
    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")
    
    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test") 
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

In [9]:
def name_model(config):
    task = "multi_task_" + "_".join(config["currency_list"]) if len(config["currency_list"]) > 1 else "single_task_" + config["currency_list"][0]
    classification = "multi_classification" if config["n_classes"] > 2 else "binary_classification"
    trend_removed = "trend_removed" if config["remove_trend"] else ""
    loss_weighted = "loss_weighted" if config["loss_weight_calculate"] else ""

    return "_".join([task, "multi_head_attention", loss_weighted, classification, trend_removed])

In [10]:
#deneme 1 

In [11]:
CONFIG = {#fix for this project
          "window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "frenquency": "D", 
          "neutral_quantile": 0.33,
          "batch_size": 16,}

In [32]:
config = CONFIG.copy()
config.update({"n_classes": 3,
               "currency_list": ['BTC', 'ETH', 'LTC'],#["BTC"], #
               "remove_trend": True,
               "loss_weight_calculate": False, 
               "indicators": True, 
               "imfs": False,
               "ohlv": False}
             )

In [33]:
MODEL_NAME = name_model(config)

CURRENCY_LST = config["currency_list"]
N_CLASSES = config["n_classes"]

REMOVE_TREND =config["remove_trend"]
LOSS_WEIGHT_CALCULATE = config["loss_weight_calculate"]

TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = config["dataset_percentages"] 
WINDOW_SIZE = config["window_size"]
FREQUENCY = config["frenquency"]
NEUTRAL_QUANTILE = config["neutral_quantile"] if N_CLASSES > 2 else 0 
BATCH_SIZE= config["batch_size"]
INDICATORS = config["indicators"]
IMFS = config["imfs"]
OHLV = config["ohlv"]
#####

In [34]:
X, y, features, dfs = get_data(CURRENCY_LST,
                            N_CLASSES,
                             FREQUENCY, 
                             WINDOW_SIZE,
                             neutral_quantile = NEUTRAL_QUANTILE,
                             log_price=True,
                             remove_trend=REMOVE_TREND,
                             include_indicators = INDICATORS,
                             include_imfs = IMFS, 
                               open_high_low_volume = OHLV
                                )
INPUT_FEATURE_SIZE = X.shape[-1]

train_dataset, val_dataset, test_dataset = [TimeSeriesDataset(CURRENCY_LST, 
                                                              X, 
                                                              y, 
                                                              dtype, 
                                                              TRAIN_PERCENTAGE, 
                                                              VAL_PERCENTAGE, 
                                                              TEST_PERCENTAGE, 
                                                              WINDOW_SIZE) for dtype in ['train', 'val', 'test']]
config["dataset_sizes"] = [len(train_dataset), len(val_dataset), len(test_dataset)]

In [35]:
X.shape

(3, 1235, 84)

In [36]:
wandb.init(project="deneme",
           config=config,
           name = MODEL_NAME)
logger = WandbLogger()
# logger = TensorBoardLogger("../output/models/lstm_model_logs", name="self_attention")
model = TradePredictor( 
                    input_dim=INPUT_FEATURE_SIZE,
                    model_dim=64,
                    num_heads=8,
                    num_classes=N_CLASSES,
                    num_layers=4,
                    dropout=0.5,
                    lr=5e-4,
                    train_dataset = train_dataset,
                    val_dataset = val_dataset,
                    test_dataset = test_dataset,
                    calculate_loss_weights = LOSS_WEIGHT_CALCULATE,
                    currencies = CURRENCY_LST,
                    window_size = WINDOW_SIZE,
                    batch_size=BATCH_SIZE,)

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.003,
   patience=25,
   verbose=True,
   mode='min'
)
trainer = pl.Trainer(#default_root_dir=root_dir, 
                     #checkpoint_callback=ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                     gpus=-1 , 
                     max_epochs=80,
                     gradient_clip_val=2,
                     progress_bar_refresh_rate=1, 
                     logger = logger, 
                     callbacks=[early_stop_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [40]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Exception: The wandb backend process has shutdown

In [38]:
trainer.test()
wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.2142857164144516,
 'BTC_test_f1': 0.16428573429584503,
 'ETH_test_acc': 0.4285714328289032,
 'ETH_test_f1': 0.1991342157125473,
 'LTC_test_acc': 0.3928571343421936,
 'LTC_test_f1': 0.18792973458766937,
 'test_loss': 1.1019856929779053}
--------------------------------------------------------------------------------


train_loss_step,1.0889
epoch,21
trainer/global_step,1551
_runtime,107
_timestamp,1625869569
_step,73
BTC_train_acc,0.35944
BTC_train_f1,0.30531
ETH_train_acc,0.35422
ETH_train_f1,0.30334
LTC_train_acc,0.32724


train_loss_step,▁█▅▄▄▄▅▄▄▄▄▃▄▃▃▄▄▃▃▃▃▄▅▄▄▄▃▃▃▄▃
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
BTC_train_acc,▁▆▆▄▄▁▅▆▄▅▅▅▄▆█▇▇▇▅▅▆
BTC_train_f1,▃█▆▅▅▂▄▄▃▅▂▄▃▁██▃▆▃▂▆
ETH_train_acc,▇▁▂▄▅▆▆█▅▇▇▄▄▅▆▃▄▆▄▆▇
ETH_train_f1,█▃▄▆▆▆▅▅▄▇▄▄▃▁▆▃▂▅▄▃▆
LTC_train_acc,▄▃▅█▂▅▄▂▂▂▂▆▃▅▆▅▅▃▁▃▃


In [5]:
#denemeleri tamamlandı experiment_mha.py dosyasına yazıldı işlemler
def experiment(script):
    !python ../pipelines/multi_task_price_change_prediction/experiment_mha.py $script

In [6]:
CONFIG = {#fix for this project
          "window_size": 50, 
          "dataset_percentages": [0.97, 0.007, 0.023],
          "frenquency": "D", 
          "neutral_quantile": 0.33,
          "batch_size": 16, 
    "imfs": False, 
    "ohlv": False, 
    "remove_trend": True}

In [7]:
from sklearn.model_selection import ParameterGrid
param_grid = {
          "n_classes": [2,3],
          "currency_list": [['BTC'], ['ETH'], ['LTC'],  ['BTC', 'ETH', 'LTC']],
          "loss_weight_calculate": [True, False], 
        "indicators": [True]}

In [8]:
for c in (ParameterGrid(param_grid)):
    config = CONFIG.copy()
    config.update(c)
    script = "--currency-list " + " ".join([i for i in config["currency_list"]])
    script += " -trend " + str(1 if config["remove_trend"] else 0)
    script += " -ohlv " + str(1 if config["ohlv"] else 0)
    script += " -imfs " + str(1 if config["imfs"] else 0)
    script += " -classes " + str(config["n_classes"] )
    script += " -weight " + str(1 if config["loss_weight_calculate"] else 0) 
    script += " -indicators " + str(1 if config["indicators"] else 0)
    print (script)
    experiment(script)

--currency-list BTC -trend 1 -ohlv 0 -imfs 0 -classes 2 -weight 1 -indicators 1
/home/aysenurk/anaconda3/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/aysenurk/anaconda3/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
wandb: Currently logged in as: aysenurk (use `wandb login --relogin` to force relogin)
2021-07-10 11:02:16.007236: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: Tracking run with wandb version 0.10.33
wandb: Syncing run single_task_BTC_multi_head_attention_binary_classification
wandb:  View project at https://wandb.ai/aysenurk/price_change_v3
wandb:  View run at https://wandb.ai/aysenurk/price_change_v3/runs/3l8i5ou7
wandb: Run data is saved locally in /home/aysenurk/Projects/OzU/mu

Epoch 16:  95%|▉| 20/21 [00:00<00:00, 31.18it/s, loss=0.674, v_num=5ou7, BTC_val
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved by 0.009 >= min_delta = 0.003. New best score: 0.678
Epoch 16: 100%|█| 21/21 [00:00<00:00, 31.61it/s, loss=0.674, v_num=5ou7, BTC_val
Epoch 17:  95%|▉| 20/21 [00:00<00:00, 30.33it/s, loss=0.676, v_num=5ou7, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 17: 100%|█| 21/21 [00:00<00:00, 30.93it/s, loss=0.676, v_num=5ou7, BTC_val
Epoch 18:  95%|▉| 20/21 [00:00<00:00, 29.45it/s, loss=0.662, v_num=5ou7, BTC_val
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved by 0.011 >= min_delta = 0.003. New best score: 0.668
Epoch 18: 100%|█| 21/21 [00:00<00:00, 29.86it/s, loss=0.662, v_num=5ou7, BTC_val
Epoch 19:  95%|▉| 20/21 [00:00<00:00, 31.06it/s, loss=0.646, v_num=5ou7, BTC_val
Validating: 0it [00:00, 


wandb: Waiting for W&B process to finish, PID 108605
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_110214-3l8i5ou7/logs/debug.log
wandb: Find internal logs for this run at: /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_110214-3l8i5ou7/logs/debug-internal.log
wandb: Run summary:
wandb:         BTC_train_acc 0.79554
wandb:          BTC_train_f1 0.79157
wandb:      train_loss_epoch 0.46037
wandb:                 epoch 49
wandb:   trainer/global_step 1000
wandb:              _runtime 42
wandb:            _timestamp 1625904176
wandb:                 _step 120
wandb:           BTC_val_acc 0.61538
wandb:            BTC_val_f1 0.60606
wandb:              val_loss 0.86962
wandb:       train_loss_step 0.62243
wandb:          BTC_test_acc 0.63636


Epoch 7:  95%|▉| 20/21 [00:00<00:00, 31.25it/s, loss=1.06, v_num=qsii, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 7: 100%|█| 21/21 [00:00<00:00, 31.47it/s, loss=1.06, v_num=qsii, BTC_val_a
Epoch 8:  95%|▉| 20/21 [00:00<00:00, 31.93it/s, loss=1.07, v_num=qsii, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 8: 100%|█| 21/21 [00:00<00:00, 32.53it/s, loss=1.07, v_num=qsii, BTC_val_a
Epoch 9:  95%|▉| 20/21 [00:00<00:00, 30.35it/s, loss=1.07, v_num=qsii, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 9: 100%|█| 21/21 [00:00<00:00, 30.90it/s, loss=1.07, v_num=qsii, BTC_val_a
Epoch 10:  95%|▉| 20/21 [00:00<00:00, 31.23it/s, loss=1.05, v_num=qsii, BTC_val_
Validating: 0it [00:00, ?it/s]
Epoch 10: 100%|█| 21/21 [00:00<00:00, 31.85it/s, loss=1.05, v_num=qsii, BTC_val_
Epoch 11:  95%|▉| 20/21 [00:00<00:00, 30.83it/s, loss=1.07, v_num=qsii, BTC_val_
Validating: 0it [00:00, ?it/s]
Epoch 11: 100%|█| 21/21 [00:00<00:00, 31.42it/s, loss=1.07, v_num=qsii, BTC_val_
Epoch 12:  95%|▉| 20/21 [00:00<00:0

Epoch 43:  95%|▉| 20/21 [00:00<00:00, 30.63it/s, loss=0.771, v_num=qsii, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 43: 100%|█| 21/21 [00:00<00:00, 31.19it/s, loss=0.771, v_num=qsii, BTC_val
Epoch 44:  95%|▉| 20/21 [00:00<00:00, 31.28it/s, loss=0.778, v_num=qsii, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 44: 100%|█| 21/21 [00:00<00:00, 31.82it/s, loss=0.778, v_num=qsii, BTC_val
Epoch 45:  95%|▉| 20/21 [00:00<00:00, 31.56it/s, loss=0.796, v_num=qsii, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 45: 100%|█| 21/21 [00:00<00:00, 32.07it/s, loss=0.796, v_num=qsii, BTC_val
Epoch 46:  95%|▉| 20/21 [00:00<00:00, 30.68it/s, loss=0.764, v_num=qsii, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 46: 100%|█| 21/21 [00:00<00:00, 31.33it/s, loss=0.764, v_num=qsii, BTC_val
Epoch 47:  95%|▉| 20/21 [00:00<00:00, 31.89it/s, loss=0.781, v_num=qsii, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 47: 100%|█| 21/21 [00:00<00:00, 32.45it/s, loss=0.781, v_num=qsii, BTC_val
Epoch 48:  95%|▉| 20/21 [00:00<00:0

...

Epoch 17:  95%|▉| 18/19 [00:01<00:00, 15.05it/s, loss=0.598, v_num=pho1, BTC_val
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved by 0.107 >= min_delta = 0.003. New best score: 0.523
Epoch 17: 100%|█| 19/19 [00:01<00:00, 15.21it/s, loss=0.598, v_num=pho1, BTC_val
Epoch 18:  95%|▉| 18/19 [00:01<00:00, 15.31it/s, loss=0.559, v_num=pho1, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 18: 100%|█| 19/19 [00:01<00:00, 15.56it/s, loss=0.559, v_num=pho1, BTC_val
Epoch 19:  95%|▉| 18/19 [00:01<00:00, 14.51it/s, loss=0.545, v_num=pho1, BTC_val
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved by 0.036 >= min_delta = 0.003. New best score: 0.486
Epoch 19: 100%|█| 19/19 [00:01<00:00, 14.68it/s, loss=0.545, v_num=pho1, BTC_val
Epoch 20:  95%|▉| 18/19 [00:01<00:00, 15.80it/s, loss=0.537, v_num=pho1, BTC_val
Validating: 0it [00:00, 

Testing: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 38.41it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'BTC_test_acc': 0.699999988079071,
 'BTC_test_f1': 0.6914286017417908,
 'ETH_test_acc': 0.7666666507720947,
 'ETH_test_f1': 0.7599999904632568,
 'LTC_test_acc': 0.7666666507720947,
 'LTC_test_f1': 0.7599999904632568,
 'test_loss': 0.5748680830001831}
--------------------------------------------------------------------------------

wandb: Waiting for W&B process to finish, PID 112463
wandb: Program ended successfully.
wandb:                                                                                
wandb: Find user logs for this run at: /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_111924-3onzpho1/logs/debug.log
wandb: Find internal logs for this run at: /home/aysenurk/Projects/OzU/multi_task_price_change_prediction/notebooks/wandb/run-20210710_111924

Epoch 0:  95%|▉| 18/19 [00:01<00:00, 16.26it/s, loss=1.16, v_num=0scr, BTC_val_a
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Metric val_loss improved. New best score: 1.072
Epoch 0: 100%|█| 19/19 [00:01<00:00, 16.50it/s, loss=1.16, v_num=0scr, BTC_val_a
                                                                                /home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  warning_cache.deprecation(
Epoch 1:  95%|▉| 18/19 [00:01<00:00, 16.20it/s, loss=1.15, v_num=0scr, BTC_val_a
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|█| 19/19 [00:01<00:00, 16.49it/s, loss=1.15, v_num=0scr, BTC_val_a
Epoch

Epoch 31:  95%|▉| 18/19 [00:01<00:00, 14.77it/s, loss=0.798, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 31: 100%|█| 19/19 [00:01<00:00, 15.10it/s, loss=0.798, v_num=0scr, BTC_val
Epoch 32:  95%|▉| 18/19 [00:01<00:00, 15.12it/s, loss=0.772, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 32: 100%|█| 19/19 [00:01<00:00, 15.37it/s, loss=0.772, v_num=0scr, BTC_val
Epoch 33:  95%|▉| 18/19 [00:01<00:00, 15.01it/s, loss=0.783, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 33: 100%|█| 19/19 [00:01<00:00, 15.32it/s, loss=0.783, v_num=0scr, BTC_val
Epoch 34:  95%|▉| 18/19 [00:01<00:00, 15.15it/s, loss=0.768, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 34: 100%|█| 19/19 [00:01<00:00, 15.27it/s, loss=0.768, v_num=0scr, BTC_val
Epoch 35:  95%|▉| 18/19 [00:01<00:00, 14.86it/s, loss=0.772, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 35: 100%|█| 19/19 [00:01<00:00, 15.13it/s, loss=0.772, v_num=0scr, BTC_val
Epoch 36:  95%|▉| 18/19 [00:01<00:0

Epoch 69:  95%|▉| 18/19 [00:01<00:00, 15.03it/s, loss=0.687, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 69: 100%|█| 19/19 [00:01<00:00, 15.22it/s, loss=0.687, v_num=0scr, BTC_val
Epoch 70:  95%|▉| 18/19 [00:01<00:00, 14.78it/s, loss=0.695, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Epoch 70: 100%|█| 19/19 [00:01<00:00, 15.04it/s, loss=0.695, v_num=0scr, BTC_val
Epoch 71:  95%|▉| 18/19 [00:01<00:00, 14.63it/s, loss=0.689, v_num=0scr, BTC_val
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                         | 0/1 [00:00<?, ?it/s]Monitored metric val_loss did not improve in the last 25 records. Best score: 0.721. Signaling Trainer to stop.
Epoch 71: 100%|█| 19/19 [00:01<00:00, 14.93it/s, loss=0.689, v_num=0scr, BTC_val
Epoch 71: 100%|█| 19/19 [00:01<00:00, 14.89it/s, loss=0.689, v_num=0scr, BTC_val
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWa